# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff6c4c9b9a0>
├── 'a' --> tensor([[-1.0137, -1.5753, -1.5998],
│                   [-1.7802,  1.4487, -0.4156]])
└── 'x' --> <FastTreeValue 0x7ff6c4c9b820>
    └── 'c' --> tensor([[ 0.0316,  1.0273, -1.2041,  2.1715],
                        [-2.3469,  0.2098,  0.2584,  0.2096],
                        [-1.1947,  1.1794,  1.1791,  0.1100]])

In [4]:
t.a

tensor([[-1.0137, -1.5753, -1.5998],
        [-1.7802,  1.4487, -0.4156]])

In [5]:
%timeit t.a

58.2 ns ± 0.0659 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff6c4c9b9a0>
├── 'a' --> tensor([[-0.5715, -2.5684,  0.1855],
│                   [ 0.5856, -1.4528, -0.0561]])
└── 'x' --> <FastTreeValue 0x7ff6c4c9b820>
    └── 'c' --> tensor([[ 0.0316,  1.0273, -1.2041,  2.1715],
                        [-2.3469,  0.2098,  0.2584,  0.2096],
                        [-1.1947,  1.1794,  1.1791,  0.1100]])

In [7]:
%timeit t.a = new_value

57.7 ns ± 0.0214 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.0137, -1.5753, -1.5998],
               [-1.7802,  1.4487, -0.4156]]),
    x: Batch(
           c: tensor([[ 0.0316,  1.0273, -1.2041,  2.1715],
                      [-2.3469,  0.2098,  0.2584,  0.2096],
                      [-1.1947,  1.1794,  1.1791,  0.1100]]),
       ),
)

In [10]:
b.a

tensor([[-1.0137, -1.5753, -1.5998],
        [-1.7802,  1.4487, -0.4156]])

In [11]:
%timeit b.a

57.4 ns ± 0.0443 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.4485, -1.0035, -0.1931],
               [ 1.4863,  0.9830,  0.6620]]),
    x: Batch(
           c: tensor([[ 0.0316,  1.0273, -1.2041,  2.1715],
                      [-2.3469,  0.2098,  0.2584,  0.2096],
                      [-1.1947,  1.1794,  1.1791,  0.1100]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.215 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

828 ns ± 0.133 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 19 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 282 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff6a01b9cd0>
├── 'a' --> tensor([[[-1.0137, -1.5753, -1.5998],
│                    [-1.7802,  1.4487, -0.4156]],
│           
│                   [[-1.0137, -1.5753, -1.5998],
│                    [-1.7802,  1.4487, -0.4156]],
│           
│                   [[-1.0137, -1.5753, -1.5998],
│                    [-1.7802,  1.4487, -0.4156]],
│           
│                   [[-1.0137, -1.5753, -1.5998],
│                    [-1.7802,  1.4487, -0.4156]],
│           
│                   [[-1.0137, -1.5753, -1.5998],
│                    [-1.7802,  1.4487, -0.4156]],
│           
│                   [[-1.0137, -1.5753, -1.5998],
│                    [-1.7802,  1.4487, -0.4156]],
│           
│                   [[-1.0137, -1.5753, -1.5998],
│                    [-1.7802,  1.4487, -0.4156]],
│           
│                   [[-1.0137, -1.5753, -1.5998],
│                    [-1.7802,  1.4487, -0.4156]]])
└── 'x' --> <FastTreeValue 0x7ff6a01b9b50>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 44 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff6a01b99a0>
├── 'a' --> tensor([[-1.0137, -1.5753, -1.5998],
│                   [-1.7802,  1.4487, -0.4156],
│                   [-1.0137, -1.5753, -1.5998],
│                   [-1.7802,  1.4487, -0.4156],
│                   [-1.0137, -1.5753, -1.5998],
│                   [-1.7802,  1.4487, -0.4156],
│                   [-1.0137, -1.5753, -1.5998],
│                   [-1.7802,  1.4487, -0.4156],
│                   [-1.0137, -1.5753, -1.5998],
│                   [-1.7802,  1.4487, -0.4156],
│                   [-1.0137, -1.5753, -1.5998],
│                   [-1.7802,  1.4487, -0.4156],
│                   [-1.0137, -1.5753, -1.5998],
│                   [-1.7802,  1.4487, -0.4156],
│                   [-1.0137, -1.5753, -1.5998],
│                   [-1.7802,  1.4487, -0.4156]])
└── 'x' --> <FastTreeValue 0x7ff6a01b9670>
    └── 'c' --> tensor([[ 0.0316,  1.0273, -1.2041,  2.1715],
                        [-2.3469,  0.2098,  0.2584,  0.2096],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 29.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.6 µs ± 229 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.0137, -1.5753, -1.5998],
                [-1.7802,  1.4487, -0.4156]],
       
               [[-1.0137, -1.5753, -1.5998],
                [-1.7802,  1.4487, -0.4156]],
       
               [[-1.0137, -1.5753, -1.5998],
                [-1.7802,  1.4487, -0.4156]],
       
               [[-1.0137, -1.5753, -1.5998],
                [-1.7802,  1.4487, -0.4156]],
       
               [[-1.0137, -1.5753, -1.5998],
                [-1.7802,  1.4487, -0.4156]],
       
               [[-1.0137, -1.5753, -1.5998],
                [-1.7802,  1.4487, -0.4156]],
       
               [[-1.0137, -1.5753, -1.5998],
                [-1.7802,  1.4487, -0.4156]],
       
               [[-1.0137, -1.5753, -1.5998],
                [-1.7802,  1.4487, -0.4156]]]),
    x: Batch(
           c: tensor([[[ 0.0316,  1.0273, -1.2041,  2.1715],
                       [-2.3469,  0.2098,  0.2584,  0.2096],
                       [-1.1947,  1.1794,  1.1791,  0.1100]],
         

In [26]:
%timeit Batch.stack(batches)

78.2 µs ± 455 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.0137, -1.5753, -1.5998],
               [-1.7802,  1.4487, -0.4156],
               [-1.0137, -1.5753, -1.5998],
               [-1.7802,  1.4487, -0.4156],
               [-1.0137, -1.5753, -1.5998],
               [-1.7802,  1.4487, -0.4156],
               [-1.0137, -1.5753, -1.5998],
               [-1.7802,  1.4487, -0.4156],
               [-1.0137, -1.5753, -1.5998],
               [-1.7802,  1.4487, -0.4156],
               [-1.0137, -1.5753, -1.5998],
               [-1.7802,  1.4487, -0.4156],
               [-1.0137, -1.5753, -1.5998],
               [-1.7802,  1.4487, -0.4156],
               [-1.0137, -1.5753, -1.5998],
               [-1.7802,  1.4487, -0.4156]]),
    x: Batch(
           c: tensor([[ 0.0316,  1.0273, -1.2041,  2.1715],
                      [-2.3469,  0.2098,  0.2584,  0.2096],
                      [-1.1947,  1.1794,  1.1791,  0.1100],
                      [ 0.0316,  1.0273, -1.2041,  2.1715],
                      [-2.3469,  

In [28]:
%timeit Batch.cat(batches)

142 µs ± 350 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

305 µs ± 1.93 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
